In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc

import Qubits

In [6]:
transmonC = Qubits.transmon_charge(20, 0.24, 10, 0)#Values given in Ghz*hbar

transmonC.solve()

GammaC = transmonC.T_1_noise()

print(1/(GammaC))
print(1/(GammaC)*1e6)

165.88389198653636
165883891.98653635


In [7]:
transmonF = Qubits.transmon_flux(200, 0.24, 10, 0)#Values given in Ghz*hbar

transmonF.solve()

GammaF = transmonF.T_1_noise()

print(1/(GammaF))
print(1/(GammaF)*1e6)

1.5743929731418218e+27
1.5743929731418218e+33


In [8]:
gamma_arr = transmonC.T_1_gamma()
print(gamma_arr)

print(1/gamma_arr*1e6)


print(1/np.sum(gamma_arr)*1e6)

[15.07078216 17.5747128 ]
[66353.55679461 56899.93408425]
30632.0979709992


In [9]:
gamma_arr_flux = transmonF.T_1_gamma()
print(gamma_arr_flux)

print(1/gamma_arr_flux*1e6)
print(1/np.sum(gamma_arr_flux)*1e6)

[15.05742487 17.55263253]
[66412.41837502 56971.511155  ]
30665.386073334423
